In [163]:
# I import the auction price data and create an average price by date for each size 
# I look at the transfers database
import os 
pwd="C:/Users/sarah/Dropbox/Upenn Year 3/paper/IP Address/0_Old/python"
os.chdir(pwd)
os.getcwd()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from datetime import datetime
arin4=pd.read_csv('C:/Users/sarah/Dropbox/Upenn Year 3/paper/IP Address/arin data/data/arin4_all.csv')
transfers=pd.read_csv('8_transfers_cidr_updated.csv')
transfers=transfers.drop(columns='type')
transfers['transfer_date']=pd.to_datetime(transfers['transfer_date'])
transfers['transfer_date']=transfers['transfer_date'].dt.date
transfers['transfer_cidr']=transfers['transfer_cidr'].astype(str)
transfers['size2']=transfers['transfer_cidr'].str[-2:]
transfers['transfer_cidr']=transfers['transfer_cidr'].str[:-3]
transfers.loc[transfers['size2']=='11','size']=2097152
transfers.loc[transfers['size2']=='12','size']=1048576
transfers.loc[transfers['size2']=='13','size']=524288
transfers.loc[transfers['size2']=='14','size']=262144
transfers.loc[transfers['size2']=='15','size']=131072
transfers.loc[transfers['size2']=='16','size']=65536
transfers.loc[transfers['size2']=='17','size']=32768
transfers.loc[transfers['size2']=='18','size']=16384
transfers.loc[transfers['size2']=='19','size']=8192
transfers.loc[transfers['size2']=='20','size']=4096
transfers.loc[transfers['size2']=='21','size']=2048
transfers.loc[transfers['size2']=='22','size']=1024
transfers.loc[transfers['size2']=='23','size']=512
transfers.loc[transfers['size2']=='10','size']=4194304
transfers.loc[transfers['size2']=='9','size']=8388608
transfers.loc[transfers['size2']=='24','size']=256
#transfers=pd.read_csv('4_transfers_Final_New.csv')

transfers.to_csv(r'C:/Users/sarah/Dropbox/Upenn Year 3/paper/IP Address/arin data/data/transfers.csv',index=False)
auction=pd.read_excel('C:/Users/sarah/Dropbox/Upenn Year 3/paper/IP Address/auction_data.xlsx')
auction=auction.iloc[:,0:14]
auction=auction[auction['Multi']!=1.0]
auction=auction[auction['Closing Date']!="Cancelled"]
#arin4=pd.read_csv('C:/Users/sarah/Dropbox/Upenn Year 3/paper/IP Address/arin data/data/arin4.csv')
auction.to_csv(r'C:/Users/sarah/Dropbox/Upenn Year 3/paper/IP Address/arin data/data/auction.csv',index=False)


In [164]:
auction_avg_price=auction.groupby(['Contract/ Auction Date','Size'])['Price/IP2'].mean()
auction_avg_price=pd.DataFrame(auction_avg_price)
auction_avg_price=auction_avg_price.reset_index()
auction_avg_price=auction_avg_price.rename(columns={'Contract/ Auction Date':"date_assigned",'Size':'size'})
auction_avg_price['size']=np.where(auction_avg_price['size']=="(2) /16",'16',auction_avg_price['size'])
auction_avg_price['size']=np.where(auction_avg_price['size'].str.len()>2,16,auction_avg_price['size'])
auction_avg_price['size']=auction_avg_price['size'].astype(int)
auction_avg_price.to_csv(r'C:/Users/sarah/Dropbox/Upenn Year 3/paper/IP Address/arin data/data/auction_avg_price.csv')

In [165]:
#Merge the ipv4 assignments and the resource transfers
merge=arin4.merge(transfers,how='outer',left_on=['address','date_assigned'],right_on=['transfer_cidr','transfer_date'],indicator=True)
merge['transfer']=np.where(merge['_merge']=='left_only',0,1)
merge['date_assigned']=np.where(merge['date_assigned'].isna(),merge['transfer_date'],merge['date_assigned'])
merge['date_assigned']=pd.to_datetime(merge['date_assigned'])
merge['size_x']=np.where(merge['size_x'].isna(),merge['size_y'],merge['size_x'])
merge['size_x']=merge['size_x'].astype(int)
merge=merge.rename(columns={'size_x':'size'})
#Merge the price data into ipv4 and ipv6 database
merge.to_csv(r'C:/Users/sarah/Dropbox/Upenn Year 3/paper/IP Address/arin data/data/ipv4.csv')

In [205]:
#I still have to merge these databases with price, I can't merge for some reason
merge=merge.sort_values(['date_assigned','size'])
merge2=merge[merge['date_assigned']>'2018-10-15']
auction_avg_price2=auction_avg_price[auction_avg_price['date_assigned']>'2018-10-15']
auction_avg_price2=auction_avg_price2.sort_values(['date_assigned','size'])
a=pd.merge_asof(left=merge2,right=auction_avg_price2,on='date_assigned',by='size',tolerance=pd.Timedelta(days=16),direction='backward')
a[a['_merge']=='both']


,rir,country,address,size,date_assigned,g,org,prefix,source_organization.name,source_organization.country_code,...,original_cidr_3,original_cidr_4,original_cidr_5,original_cidr_6,original_cidr_7,size2,size_y,_merge,transfer,Price/IP2
0,arin,US,23.154.96.0,256,2018-10-16,assigned,f28fd689a36c6903da5411487295457a,24.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,NaN
1,apnic,HK,103.131.34.0,256,2018-10-16,assigned,A913270F,24.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,NaN
2,arin,CA,23.154.32.0,256,2018-10-16,assigned,7407f70a5e769db1ff995e195328dcc5,24.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,NaN
3,apnic,ID,103.131.29.0,256,2018-10-16,assigned,A9267445,24.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,NaN
4,lacnic,BR,192.83.207.0,256,2018-10-16,assigned,101898,24.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,NaN
5,lacnic,BR,45.162.202.0,512,2018-10-16,assigned,294260,23.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,NaN
6,apnic,AU,103.131.40.0,512,2018-10-16,assigned,A916C643,23.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,NaN
7,apnic,SG,103.131.32.0,512,2018-10-16,assigned,A915CA20,23.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,NaN
8,apnic,NZ,103.131.30.0,512,2018-10-16,assigned,A91ABDDC,23.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,NaN
9,apnic,AU,103.131.42.0,512,2018-10-16,assigned,A916A505,23.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,NaN
